<div style="background-color: #EDF7FF; border-color: #7C9DBF; border-left: 5px solid #7C9DBF; padding: 0.5em;">
 

<strong>Objectives:</strong>

    - Estudiar y explorar las series correspondientes al producto 68
    - Comprobar las estacionalidades
    - Descomponer la serie
    - Obtener la media por día de la semana, mes, etc.
    - Estudiar la asignación de missings en ventas y stock
    
    
<strong>Conclusions:</strong>

   
</div>

In [1]:
import os, sys

def get_root_path(search_word):
    """ Búsqueda hacia atrás de un directorio con nombre search_word. """
    cwd = os.getcwd()
    while str(os.getcwd()).split('/')[-1] != search_word:
        os.chdir("..")
        cwd = os.getcwd()
    return cwd

ROOT_DIR = get_root_path('TFM_StockForecast')

sys.path.insert(0, ROOT_DIR) # Poner en el primer lugar del PATH (evitar paquetes con mismos nombres en otros directorios)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.load_data import load_csv, load_data
from src.prepare_data import prepare_train_data

In [2]:
from statsmodels.tsa.seasonal import seasonal_decompose

def plot_seasonal_descompose(serie, period):
    
    result = seasonal_decompose(serie, model='additive', period=period)
    
    plt.figure(figsize=(20,40))
    plt.subplot(4,1,1)
    result.observed.plot(title='Observed')
    plt.subplot(4,1,2)
    result.trend.plot(title='Trend')
    plt.subplot(4,1,3)
    result.seasonal.plot(title='Seasonal - Period {}'.format(period))
    plt.subplot(4,1,4)
    result.resid.plot(title='Residual')
    plt.tight_layout()

In [3]:
df = load_data()
df.head()

===================  LOAD DATASET stock ====================
Reading CSV in data/raw/03_TablaStock.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 628.
============  CLEAN STOCK  =============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). Rows dropped: 1784
[WARNING] Number of dates missings:                                83
[WARNING] Number of dates missings (droping sundays and festivos): 63
[WARNING] Stock data with different units for same product & data. Rows: 12666
Dataset stock loaded. Shape: (25518, 3)

===================  LOAD DATASET ventas ===================
Reading CSV in data/raw/01_TablaVentas.csv...
=============  CLEAN CSV  ==============
Set columns to lowcase.
Sort values by date.
Dropped duplicates. Rows dropped: 0.
============  CLEAN VENTAS  ============
Dropped rows with data from products with no stock data or out-of-date (descatalogados). 

/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ts['missing'] = (ts.udsstock.isna()) * 1


Getting datetime variables: weekday, quarter, month, weekofyear
Getting deltaStock as the difference of Stock from today to tomorrow


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_data['deltaStock'] = prod_data.udsstock.diff(periods=-1)


Getting rolling windows of last 5 days by product and weekday for column udsventa


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data[colname] = day_data[col].rolling(4, win_type='triang', min_periods=1).mean()
/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data["meanwd_"+col] = day_data[col].mean()


Getting rolling windows of last 5 days by product and weekday for column udsstock
Getting rolling windows of last 5 days by product and weekday for column udsprevisionempresa


/mnt/c/Users/Administrador/Documents/GitHub/TFM_StockForecast/src/create_variables.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prod_data['udsprevision_' + str(period)] = data['udsprevisionempresa'].shift(periods=period, fill_value=0)


Created new variable with the prevision ventas shifted 1 period/s.
Created new variable with the prevision ventas shifted 2 period/s.
Created new variable with the prevision ventas shifted 3 period/s.
Created new variable with the prevision ventas shifted 4 period/s.
Created new variable with the prevision ventas shifted 5 period/s.
Created new variable with the prevision ventas shifted 6 period/s.
Created new variable with the prevision ventas shifted 7 period/s.


,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,...,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa,udsprevision_1,udsprevision_2,udsprevision_3,udsprevision_4,udsprevision_5,udsprevision_6,udsprevision_7
0,2019-06-05,1,18275.0,2833.0,102304.0,0.0,0.0,0.0,2,2,...,11012.218750,102304.0,37273.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-06-05,10,2267.0,1874.0,36888.0,1.0,0.0,0.0,2,2,...,1822.161290,36888.0,10102.540541,102304.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-06-05,11,2506.0,2755.0,31078.0,0.0,0.0,0.0,2,2,...,3987.666667,31078.0,10973.815789,36888.0,102304.0,0.0,0.0,0.0,0.0,0.0
3,2019-06-05,12,1279.0,1161.0,33661.0,0.0,0.0,0.0,2,2,...,1605.741935,33661.0,5724.236842,31078.0,36888.0,102304.0,0.0,0.0,0.0,0.0
4,2019-06-05,13,2493.0,1603.0,32119.0,1.0,0.0,0.0,2,2,...,3113.843750,32119.0,8500.421053,33661.0,31078.0,36888.0,102304.0,0.0,0.0,0.0


In [4]:
data = df.loc[(df.producto=='68')]
data.head()

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,...,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa,udsprevision_1,udsprevision_2,udsprevision_3,udsprevision_4,udsprevision_5,udsprevision_6,udsprevision_7
50,2019-06-05,68,1545.0,392.0,7451.0,0.0,0.0,0.0,2,2,...,1007.093750,7451.0,1753.947368,15463.0,5904.0,13529.0,7269.0,6873.0,14448.0,4255.0
122,2019-06-06,68,NaN,428.0,9499.0,0.0,0.0,2.0,3,2,...,964.343750,9499.0,2238.692308,11050.0,8555.0,15672.0,9918.0,8285.0,11535.0,7714.0
194,2019-06-07,68,NaN,510.0,5106.0,0.0,0.0,2.0,4,2,...,794.533333,5106.0,1444.564103,17639.0,8253.0,17105.0,8171.0,9307.0,14816.0,2344.0
266,2019-06-08,68,NaN,81.0,1735.0,0.0,0.0,2.0,5,2,...,1140.357143,1735.0,715.736842,75.0,3250.0,3.0,1815.0,1976.0,50.0,1735.0
338,2019-06-09,68,NaN,NaN,NaN,0.0,0.0,2.0,6,2,...,918.208333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['fecha', 'producto', 'udsstock', 'udsventa', 'udsprevisionempresa',
       'promo', 'festivo', 'stockMissingType', 'weekday', 'quarter', 'month',
       'weekofyear', 'working_day', 'sin_weekday', 'cos_weekday', 'deltaStock',
       'roll4wd_udsventa', 'meanwd_udsventa', 'roll4wd_udsstock',
       'meanwd_udsstock', 'roll4wd_udsprevisionempresa',
       'meanwd_udsprevisionempresa', 'udsprevision_1', 'udsprevision_2',
       'udsprevision_3', 'udsprevision_4', 'udsprevision_5', 'udsprevision_6',
       'udsprevision_7'],
      dtype='object')

In [10]:
data

,fecha,producto,udsstock,udsventa,udsprevisionempresa,promo,festivo,stockMissingType,weekday,quarter,...,weekofyear,deltaStock,roll4wd_udsventa,meanwd_udsventa,roll4wd_udsstock,meanwd_udsstock,roll4wd_udsprevisionempresa,meanwd_udsprevisionempresa,holiday_udsstock,udsprevision1
0,2019-06-05,68,1545.000000,392.0,7451.0,0.0,0.0,0.0,2,2,...,23,NaN,392.000,428.589744,1545.0,1007.093750,7451.00,1753.947368,1545.0,0.0
1,2019-06-06,68,964.343750,428.0,9499.0,0.0,0.0,2.0,3,2,...,23,NaN,428.000,430.425000,NaN,964.343750,9499.00,2238.692308,1429.0,7451.0
2,2019-06-07,68,794.533333,510.0,5106.0,0.0,0.0,2.0,4,2,...,23,NaN,510.000,506.641026,NaN,794.533333,5106.00,1444.564103,1429.0,9499.0
3,2019-06-08,68,1140.357143,81.0,1735.0,0.0,0.0,2.0,5,2,...,23,NaN,81.000,105.850000,NaN,1140.357143,1735.00,715.736842,1429.0,5106.0
4,2019-06-09,68,1429.000000,0.0,0.0,0.0,0.0,2.0,6,2,...,23,NaN,NaN,32.500000,NaN,918.208333,NaN,NaN,1429.0,1735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,2020-03-18,68,1019.000000,587.0,2303.0,0.0,0.0,0.0,2,1,...,12,291.0,584.875,428.589744,1314.4,1007.093750,2518.50,1753.947368,1019.0,891.0
288,2020-03-19,68,728.000000,654.0,1926.0,0.0,0.0,0.0,3,1,...,12,-363.0,517.750,430.425000,899.0,964.343750,2910.75,2238.692308,728.0,2303.0
289,2020-03-20,68,1091.000000,678.0,1190.0,0.0,0.0,0.0,4,1,...,12,-1304.0,773.000,506.641026,956.4,794.533333,2168.50,1444.564103,1091.0,1926.0
290,2020-03-21,68,2395.000000,66.0,348.0,0.0,0.0,0.0,5,1,...,12,87.0,219.875,105.850000,818.0,1140.357143,933.25,715.736842,2395.0,1190.0


In [ ]:
data = data[["udsventa","udsprevisionempresa", "udsstock"]]

data.info()

In [ ]:
actual = np.array([1,2,3,1])
err_naive = np.mean(np.diff(np.array(actual)))

In [ ]:
actual = np.array([1,2,3,1])
np.mean(np.abs(np.diff(np.array(actual))))

# Time Series

In [ ]:
subdf = df.loc[(df.producto=='68')]
for series in ['udsventa', 'udsstock', 'udsprevisionempresa']:
    subdf[[series, 'roll4wd_'+series]].plot(figsize=(20,10), title="Producto: {}".format('68'))
    plt.show()
    


In [ ]:
def _assing_missings_stock(df, col):
    """Assign missings: backfill for holiday
                        4wd rolling windows for else???
    """
    print("Assigning missings for {}".format(col))
    data = df.copy()
    holiday_data = data.loc[(data['festivo'] == 1) | (data['weekday'] == 6)]
    wd_data = data.loc[(data['festivo'] != 1) & (data['weekday'] != 6)]
    
    print("Missing in dataset:               {} ({} total rows).".format(sum(data[col].isna()), data.shape[0]))
    print("Missings in holiday days:         {} ({} total rows).".format(sum(holiday_data[col].isna()), holiday_data.shape[0]))
    print("Missings in working days:         {} ({} total rows).".format(sum(wd_data[col].isna()), wd_data.shape[0]))
    
    data['holiday_' + col] = data.fillna(method='bfill')
    print("Assigned missings for holiday data - Remaining missings:      {}".format(sum(data[col].isna())))
    
    data.loc[(data['festivo'] != 1) & (data['weekday'] != 6), col] = wd_data[col].fillna(wd_data["roll4wd_" + col])
    print("Assigned missings for working days data - Remaining missings: {}".format(sum(data[col].isna())))
    return data
col = 'udsstock'
data = subdf.copy()

# Creamos vectores de filtro
holiday = (data['festivo'] == 1) | (data['weekday'] == 6)
workday = (data['festivo'] != 1) & (data['weekday'] != 6)
missing = data[col].isna()
incrementnot0 = (~data['deltaStock'].isna()) & (data['deltaStock'] != 0)


print("Missing in dataset:               {} ({} total rows).".format(sum(missing), data.shape[0]))
print("Missings in holiday days:         {} ({} total rows).".format(sum(missing & holiday), sum(holiday)))
print("Missings in working days:         {} ({} total rows).".format(sum(missing & workday), sum(workday)))

print("In holiday days, the increment of stock was different from 0 in {} days.".format(sum(holiday & incrementnot0)))

# Creamos una columna para los holiday missings
data['holiday_missing'] = data[col].fillna(method='bfill')

# Asignamos todos los valores como rolling windows
data[col] = data[col].fillna(data["roll4wd_" + col])
print("Assigned missings for working days data - Rolling windows method.")

# Asignamos los holidays por encima y borramos la columna intermedia
data.loc[holiday & missing, col] = data.loc[holiday & missing, "holiday_missing"]
data = data.drop("holiday_missing", axis=1)
print("Assigned missings for holiday data - backfill method")
print("Assigned missings for holiday data - Remaining missings: {}".format(sum(data[col].isna())))



data


In [ ]:
subdf = df.loc[(df.producto=='1')]
subdf['deltaStock1'] = subdf.udsstock.diff(periods=1)
subdf.loc[subdf.weekday == 6]

In [ ]:
plot_seasonal_descompose(subdf.udsventa, 7)

## Missings de ventas

In [ ]:
miss[['weekday','udsventa']].fillna(0).groupby("weekday")['udsventa'].count()

In [ ]:
data = subdf
col = 'roll5wd_udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))


# Asignamos 

In [ ]:
data = subdf
col = 'udsventa'
miss = data.loc[data[col].isna()]
no_miss = data.loc[~data[col].isna()]
# Numero total de missings
print("Missings en la serie:                     {}".format(miss.shape[0]))
print("Missings en la serie sin cambio de stock: {}".format(miss.loc[miss.deltaStock == 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock > 0].shape[0]))
print("Missings en la serie con cambio de stock: {}".format(miss.loc[miss.deltaStock < 0].shape[0]))

In [ ]:
no_miss.loc[no_miss.deltaStock == 0]

In [ ]:
miss.loc[miss.deltaStock == 0]

In [ ]:
subdf.loc[subdf.weekday == 6]

In [ ]:
subdf.loc[subdf.index > '2019-08-12']

## Medias de las ventas

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].mean()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("weekday")['udsventa'].max()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].min()

In [ ]:
# Media de ventas por día de la semana
subdf.groupby("festivo")['udsventa'].max()

In [ ]:
subdf

## Ventas en festivo o domingo faltantes puede ser 0?

In [ ]:
subdf = df.loc[(df.producto=='68')]
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)] = subdf.loc[(subdf.festivo == 1) | (subdf.weekday == 6)].fillna(0)
subdf.udsventa.plot(figsize=(20,10), title="Producto: {}".format('68'))
plt.show()

In [ ]:
subdf.loc[subdf.udsventa.isna()]